<a href="https://colab.research.google.com/github/chanzae/NLP_study/blob/main/study/%5Bmetacode%5D_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%8B%A4%EC%8A%B53_Word_Embedding2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**패키지 설치**
> datasets  
> konlpy  
> tqdm  
> glove_python_binary(GloVe임베딩)  
> gensim

In [1]:
!pip install datasets
!pip install konlpy
!pip install tqdm
!pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 140 kB 34.7 MB/s 
     |████████████████████████████████| 212 kB 60.5 MB/s 
     |████████████████████████████████| 1.1 MB 64.0 MB/s 
     |████████████████████████████████| 101 kB 8.4 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 127 kB 64.2 MB/s 
     |████████████████████████████████| 271 kB 76.0 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 75.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's depe

# 데이터셋 준비
Naver Sentiment Movie Corpus(nsmc)
영화 리뷰를 별점을 기준으로 긍정(1), 부정(0)으로 구분한 데이터셋

In [2]:
from datasets import load_dataset

#nsmc 데이터셋 다운로드
dataset = load_dataset('nsmc')

train_sents = [dataset['train'][idx]['document'] for idx in range(len(dataset['train']))]
train_labels = [dataset['train'][idx]['label'] for idx in range(len(dataset['train']))]
test_sents = [dataset['test'][idx]['document'] for idx in range(len(dataset['test']))]
test_labels = [dataset['test'][idx]['label'] for idx in range(len(dataset['test']))]

Using custom data configuration default


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

네이버 맞춤법교정과 띄어쓰기 교정은 시간이 너무 오래 걸려 15만개의 데이터셋을 전처리 하는데 많은 시간이 필요하다. 간단하게 한글단어+어간추출+불용어 제거 정도의 간단한 전처리를 진행하고자 한다.

In [3]:
from konlpy.tag import Okt
from tqdm import tqdm

#stopwards 정의
stopwords = ['하다','한','에','와','자','과','걍','잘','좀','는','의','가','이','은','들']

okt = Okt()

train_nsmc_corpus = []
for sent in tqdm(train_sents):
    #한글이 아닌 글자는 제거
    new_sent = sent.replace("[^가-힣ㄱ-ㅎㅏ-ㅣ ]","")
    tokens = okt.morphs(sent, stem=True) # 어간추출
    tokens = [word for word in tokens if word not in stopwords]
    
    train_nsmc_corpus.append(tokens)
    
test_nsmc_corpus = []
for sent in tqdm(test_sents):
    new_sent = sent.replace("[^가-힣ㄱ-ㅎㅏ-ㅣ ]","")
    tokens = okt.morphs(sent, stem=True) # 어간추출
    tokens = [word for word in tokens if word not in stopwords]
    
    test_nsmc_corpus.append(tokens)

100%|██████████| 50000/50000 [02:39<00:00, 312.94it/s]


# Word2Vec
Word2Vec은 이웃 단어 예측을 통해 단어의 임베딩을 학습하는 기법
CBoW기법 / Skip-Gram 기법

## CBoW
주위(Window)단어를 보고 중심 단어를 예측

In [6]:
from gensim.models import Word2Vec

cbow_model = Word2Vec(sentences = train_nsmc_corpus, size = 100, window = 5, min_count = 5, workers = 4,sg = 0)
# sg = 0(cbow) / sg = 1(skip gram)

In [7]:
# 유사한 단어 찾기
cbow_model.wv.most_similar('송강호', topn=5)

[('차승원', 0.8840539455413818),
 ('한석규', 0.8795738220214844),
 ('신들리다', 0.8790381550788879),
 ('최민수', 0.872622013092041),
 ('니콜슨', 0.8683717846870422)]

In [8]:
#사전에 없는 단어 = 에러
cbow_model.wv.most_similar('자연어처리', topn=5)

KeyError: ignored

In [9]:
# 송강호라는 단어의 벡터는 아래와 같이 생김
cbow_model.wv['송강호']  

array([-1.53857857e-01,  2.16229364e-01, -7.74740279e-02,  7.25658685e-02,
       -1.40068084e-01,  6.90574273e-02,  7.17597306e-02, -4.35633987e-01,
        1.25579312e-01, -5.33600003e-02,  7.83920139e-02, -2.20748574e-01,
       -3.27244341e-01,  7.20730200e-02,  1.24521047e-01,  7.26138651e-02,
        4.64485623e-02, -6.26302063e-02,  3.58474553e-02, -5.29955149e-01,
        1.29737675e-01,  2.54381988e-02, -9.35916305e-02,  5.73938005e-02,
        3.38320345e-01, -7.26254880e-01, -5.55273592e-01,  1.36632184e-02,
        2.82800943e-01,  7.72646144e-02,  1.71634883e-01,  5.21258079e-02,
        7.24041462e-02, -7.39141256e-02, -4.97028269e-02,  3.37595865e-02,
        5.09477973e-01, -1.20799787e-01, -9.99066457e-02,  3.56916077e-02,
       -4.34965752e-02, -1.51701361e-01, -1.56108290e-01,  1.60178199e-01,
        2.47658879e-01,  1.12319693e-01,  3.74199539e-01, -1.21586155e-02,
       -3.84286866e-02,  4.11755890e-01,  2.84779370e-02,  8.28958582e-04,
        2.11691901e-01, -

In [10]:
import numpy as np

def sentence_embedding_word2vec(model, sentence, dim = 100):
    sent_emb = np.zeros(dim)
    n = 0
    
    for word in sentence:
        #vocab에 존재하지 않는 단어의 경우 예외처리
        try:
            word_emb = model.wv[word]
            sent_emb += word_emb
            n += 1
        except:
            pass
        
    # 문장 임베딩 = 단어 임베딩의 평균
    if n != 0:
        sent_emb = sent_emb / n
    return sent_emb

train_x_cbow = []
test_x_cbow = []

for _train_sent in tqdm(train_nsmc_corpus):
    train_x_cbow.append(sentence_embedding_word2vec(cbow_model, _train_sent))
    
for _test_sent in tqdm(test_nsmc_corpus):
    test_x_cbow.append(sentence_embedding_word2vec(cbow_model, _test_sent))

100%|██████████| 50000/50000 [00:02<00:00, 21943.89it/s]


### LogisticRegression(CBoW)
Linear Regression : 실수 범위의 값으로 예측  
Logistic Regression : 0과 1 사이의 값으로 예측

In [12]:
from sklearn.linear_model import LogisticRegression
linear_model_cbow = LogisticRegression() # 긍/부정 예측 모델
linear_model_cbow.fit(train_x_cbow, train_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [13]:
train_pred = linear_model_cbow.predict(train_x_cbow)
print(f'train acc : {np.mean(train_pred == train_labels)}')
 
test_pred = linear_model_cbow.predict(test_x_cbow)
print(f'test acc : {np.mean(test_pred == test_labels)}')

train acc : 0.81026
test acc : 0.80604


## Skip-Gram
skip-gram은 중심 단어를 보고 주위 단어들을 예측  
CBoW보다는 Skip Gram의 성능이 더 좋은 편이다

In [14]:
skipgram_model = Word2Vec(sentences = train_nsmc_corpus, size = 100, window = 5, min_count = 5, workers = 4, sg = 1)

train_x_skipgram = []
test_x_skipgram = []

for _train_sent in tqdm(train_nsmc_corpus):
    train_x_skipgram.append(sentence_embedding_word2vec(skipgram_model, _train_sent))
    
for _test_sent in tqdm(test_nsmc_corpus):
    test_x_skipgram.append(sentence_embedding_word2vec(skipgram_model, _test_sent))

100%|██████████| 50000/50000 [00:02<00:00, 22694.88it/s]


In [15]:
skipgram_model.wv.most_similar('송강호',topn = 5)

[('황정민', 0.8431901335716248),
 ('안성기', 0.8427969217300415),
 ('정재영', 0.8394070863723755),
 ('최민수', 0.838266134262085),
 ('한석규', 0.8213282227516174)]

### LogisticRegression(Skip Gram)

In [16]:
from sklearn.linear_model import LogisticRegression
linear_model_sg = LogisticRegression() # 긍/부정 예측 모델
linear_model_sg.fit(train_x_skipgram, train_labels)

train_pred = linear_model_sg.predict(train_x_skipgram)
print(f'train acc : {np.mean(train_pred == train_labels)}')
 
test_pred = linear_model_sg.predict(test_x_skipgram)
print(f'test acc : {np.mean(test_pred == test_labels)}')

train acc : 0.8256266666666666
test acc : 0.8229


## GloVe
Word2Vec은 이웃한 단어의 관계만 반영, 전체 코퍼스에 대한 정보는 반영하지 않는다.
GloVe는 이웃한 단어의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 학습한다. (전체 코퍼스에 대한 정보 반영)

In [17]:
from glove import Corpus, Glove

glove_corpus = Corpus()
glove_corpus.fit(train_nsmc_corpus, window = 5)

glove_model = Glove(no_components = 100, learning_rate = 0.05) #차원, 학습률

glove_model.fit(glove_corpus.matrix, epochs = 30, no_threads = 4, verbose = True) #단어별로 윈도우가 구성되어 있는데 그걸 matrix라고 함
glove_model.add_dictionary(glove_corpus.dictionary) # glove 모델에 단어 정보 추가

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [18]:
glove_model.most_similar('송강호')

[('김혜수', 0.881009128104451),
 ('전도연', 0.8804121786246181),
 ('한석규', 0.8780865573923653),
 ('류승범', 0.8316728423660535)]

In [19]:
# 단어 사전에서 단어를 찾고 그 단어의 인덱스를 벡터 임베딩 테이블에서 찾아야함
glove_model.word_vectors[glove_model.dictionary['송강호']]

array([-0.02579551,  0.02869958,  0.01255362,  0.18846187,  0.04816122,
       -0.01016842,  0.01489717,  0.08528402, -0.03505625, -0.05611933,
       -0.0425418 ,  0.06012598, -0.0298962 ,  0.01500115, -0.0326155 ,
       -0.02075431, -0.03269861,  0.02927181, -0.19148414, -0.17612755,
       -0.09108013, -0.05376632,  0.03020687, -0.06275895, -0.07068477,
        0.05440798, -0.01966829,  0.04282706, -0.02304194,  0.24331703,
        0.11725067, -0.02331024, -0.07123421,  0.03968667,  0.02960109,
       -0.07621462,  0.11279085,  0.00508688, -0.06714305, -0.04570003,
       -0.01093843,  0.02370095,  0.17377623, -0.02625564,  0.14674947,
        0.10204565,  0.05379978, -0.09347514,  0.01270553, -0.01262959,
        0.03020644,  0.03112716,  0.21875048, -0.01812737, -0.0181722 ,
        0.03059179, -0.04632222, -0.00658186,  0.11427278, -0.11327427,
        0.07903028, -0.05267953,  0.04346801,  0.04073206,  0.00968559,
        0.31220417, -0.03044421, -0.01315586, -0.09953144, -0.03

In [35]:
def sentence_embedding_glove(model, sentence, dim = 100):
  sent_emb = np.zeros(dim)
  n = 0

  for word in sentence:
    try:
      word_emb = model.word_vectors[model.dictionary[word]]
      sent_emb += word_emb
      n += 1
    except:
      pass

  if n != 0:
    sent_emb = sent_emb / n

  return sent_emb

train_x_glove = []
test_x_glove = []

for _train_sent in tqdm(train_nsmc_corpus):
    train_x_glove.append(sentence_embedding_glove(glove_model, _train_sent))

for _test_sent in tqdm(test_nsmc_corpus):
    test_x_glove.append(sentence_embedding_glove(glove_model, _test_sent))

100%|██████████| 50000/50000 [00:01<00:00, 46261.66it/s]


### LosigticReression(GloVe)

In [31]:
linear_model_glove = LogisticRegression()
linear_model_glove.fit(train_x_glove, train_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [36]:
train_pred = linear_model_glove.predict(train_x_glove)
print(f'train_acc : {np.mean(train_pred == train_labels)}')

test_pred = linear_model_glove.predict(test_x_glove)
print(f'test_acc : {np.mean(test_pred == test_labels)}')

train_acc : 0.7849
test_acc : 0.78012


로지스틱회귀가 수렴이 제대로 되지 않아서 glove의 성능이 cbow, skip gram보다 낮게 나오고 있다. 이는 제대로 학습되지 않아서 생기는 문제인데, 수렴이 잘 되고 데이터의 양이 적으면 glove의 성능이 훨씬 좋을 것으로 보인다.

# 학습한 모델 저장 및 불러오기

In [41]:
# 저장
cbow_model.save('word2vec_cbow.model')

#불러오기
cbow_model_2 = Word2Vec.load('word2vec_cbow.model')

# 생성한 모델과 저장되어 불러온 모델이 같은지 확인
cbow_vector_1 = cbow_model.wv['송강호']
cbow_vector_2 = cbow_model_2.wv['송강호']

if np.sum(np.abs(cbow_vector_1 - cbow_vector_2)) == 0:
  print('단어 일치')

단어 일치


In [43]:
# 저장
glove_model.save('glove.model')

#불러오기
glove_model_2 = glove_model.load('glove.model')

# 생성한 모델과 저장되어 불러온 모델이 같은지 확인
glove_vector_1 = glove_model.word_vectors[glove_model.dictionary['송강호']]
glove_vector_2 = glove_model_2.word_vectors[glove_model.dictionary['송강호']]

if np.sum(np.abs(glove_vector_1 - glove_vector_2)) == 0:
  print('단어 일치')

단어 일치
